In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, GRU, Activation
from keras.layers import Dense
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from matplotlib import pyplot as plt
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import pydot
from scipy.stats import pearsonr
import keras.backend as K

Using TensorFlow backend.
C:\Users\Tara\Miniconda3\envs\blockchain\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Tara\Miniconda3\envs\blockchain\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Tara\Miniconda3\envs\blockchain\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Tara\Miniconda3\envs\bl

In [10]:
np.random.seed(5151)
url='https://raw.githubusercontent.com/NayantaraPrem/EthereumPricePrediction/master/final_dataset.csv'
df = pd.read_csv(url)
df.head(4)


,Date(UTC),UnixTimeStamp,BlockSize,AvgGasPrices,GasUsed,GasLimit,BlockTime,NetworkUtilization,TransactionFees,BlockDifficulty,...,to_tx_count_a413,toSum_a413,to_tx_count_6853,toSum_6853,to_tx_count_92ff,toSum_92ff,to_tx_count_fb32,toSum_fb32,to_tx_count_84f3,toSum_84f3
0,2015-07-30,1438214400,644,0.0,0,0,4.46,0.0,0.0,0.121,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-07-31,1438300800,582,0.0,0,0,12.58,0.0,0.0,0.603,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-08-01,1438387200,575,0.0,0,0,16.34,0.0,0.0,0.887,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-08-02,1438473600,581,0.0,0,0,16.12,0.0,0.0,1.020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
output_column = 'Price'
output_column_feature = 'Price'

X = df.drop(['Date(UTC)', 'UnixTimeStamp'], axis=1)
dates = pd.to_datetime(df['Date(UTC)'], format='%Y-%m-%d')
#drop rows with NaN values
X = X.fillna(0) 
# features = []
features =['TransactionFees','AddressCount','EthereumGTrends', 'GasUsed', 'TweetCount']
from_cols = [col for col in X.columns if 'from' in col]
to_cols = [col for col in X.columns if 'to' in col]
features = features + from_cols + to_cols
#features = get_top_correlated_features(X, 5)
#X=X.diff()
#features = get_top_correlated_features(df, d = 10) # d = Num featres
n_timesteps = 7
n_features = len(features)
n_outputs = 1 # no. of days to predict
N = len(X)
X.head()

,BlockSize,AvgGasPrices,GasUsed,GasLimit,BlockTime,NetworkUtilization,TransactionFees,BlockDifficulty,NetworkHash,EtherSupply,...,to_tx_count_a413,toSum_a413,to_tx_count_6853,toSum_6853,to_tx_count_92ff,toSum_92ff,to_tx_count_fb32,toSum_fb32,to_tx_count_84f3,toSum_84f3
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
features

In [ ]:
train_split = 0.8 #20% test, 80% train
X_train = X[0:round(N*train_split)].reset_index(drop=True)
X_test = X[round(N*train_split):].reset_index(drop=True)
N_test = X_test.shape[0]
N_train = X_train.shape[0]
dates_train = dates[0:round(N*train_split)].reset_index(drop=True)
dates_test = dates[round(N*train_split):].reset_index(drop=True)
len(X_train), len(X_test)

In [ ]:
#reformulate this to predict the 'change' in prices
# X_test_1 = X_test.loc[0]
# X_train = X_train.diff().dropna()
# X_test = X_test.diff().dropna()
# N_test = X_test.shape[0]
# N_train = X_train.shape[0]

In [ ]:
#scale training data
# input = D features. Column 4 shifted is output
input_scaler = MinMaxScaler(feature_range = (0, 1)) # was (0,1) scaled
X_train_scaled = pd.DataFrame(input_scaler.fit_transform(X_train), columns=X_train.columns)
output_scaler = MinMaxScaler(feature_range = (0, 1))
output_scaler.fit(np.array(X_train[output_column]).reshape((N_train,1))) # scaling the output column
X_train_scaled.head(5)

In [ ]:
# restructure data: model expects input data X to have shape : samples x timesteps x features
# samples = N - n_timesteps
# each sample is a n_timesteps x n_features matrix of data that is used to make a single prediction
# eg. 7 days worth of daily data if n_timesteps = 7 is used to predict the 8th day value
# Y should be samples x output dimensions
X_train_windows = []
y_train_windows = []
for i in range(N_train - n_timesteps):
    X_train_windows.append(np.array(X_train_scaled.loc[i:i+n_timesteps-1, features]))
    y_train_windows.append(np.array(X_train_scaled.loc[i+n_timesteps, X_train_scaled.columns == output_column]))

X_train_windows = np.array(X_train_windows)
y_train_windows = np.array(y_train_windows)
X_train_windows.shape, y_train_windows.shape

In [ ]:
#define the model
model = Sequential()
model.add(LSTM(
    20,
    activation='relu',
    return_sequences=True))
model.add(LSTM(10, activation='relu'))
#model.add(Activation('softmax'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_logarithmic_error')

# train model
history = model.fit(X_train_windows, y_train_windows, epochs=100, verbose=0)

In [ ]:
# test prediction

#scale the input to [0,1]
X_test_scaled = pd.DataFrame(input_scaler.fit_transform(X_test), columns=X_test.columns)

#build the windows
X_test_windows = []
y_test_windows = []
y_test_dates_list = []
for i in range(N_test - n_timesteps):
    X_test_windows.append(np.array(X_test_scaled.loc[i:i+n_timesteps-1, features]))
    y_test_windows.append(np.array(X_test_scaled.loc[i+n_timesteps, X_test_scaled.columns == output_column]))
    y_test_dates_list.append(dates_test.loc[i+n_timesteps])
    #y_test_dates_list.append(dates_test.loc[i+n_timesteps+1]) # reformulated for diff calc

X_test_windows = np.array(X_test_windows)
y_test_windows = np.array(y_test_windows)

In [ ]:
#run predictions on each window
y_pred = []
y_pred_dates = []

for X_test_window in X_test_windows:
    y_pred.append(model.predict(X_test_window.reshape(1,n_timesteps,n_features))[0])
    
y_pred = np.array(y_pred)

# rescale the predictions back to the original values
y_pred_scaled = output_scaler.inverse_transform(y_pred)
y_actual_scaled = output_scaler.inverse_transform(y_test_windows)

In [ ]:
#reformulated for diff preice calc
#recover_original_from_diff(y_pred_scaled, )

In [ ]:
plt.xticks(rotation='vertical')
plt.plot(y_test_dates_list, y_pred_scaled, color='red', label='Predicted Prices')
plt.plot(y_test_dates_list, y_actual_scaled, color='green', label='Actual Prices')
plt.legend()
plt.show()

In [ ]:
def make_metric_df(y_pred,y_true):
    y_pred.name = 'y_pred'
    y_true.name = 'y_true'
    
    df = pd.concat([y_pred,y_true],axis=1)

    df['sign_pred'] = df.y_pred.apply(np.sign)
    df['sign_true'] = df.y_true.apply(np.sign)
    df['is_correct'] = 0
    df.loc[df.sign_pred * df.sign_true > 0 ,'is_correct'] = 1 # only registers 1 when prediction was made AND it was correct
    df['is_incorrect'] = 0
    df.loc[df.sign_pred * df.sign_true < 0,'is_incorrect'] = 1 # only registers 1 when prediction was made AND it was wrong
    df['is_predicted'] = df.is_correct + df.is_incorrect
    df['result'] = df.sign_pred * df.y_true 
    return df

def calc_scorecard(df):
    scorecard = pd.Series()
    # building block metrics
    scorecard.loc['accuracy'] = df.is_correct.sum()*1. / (df.is_predicted.sum()*1.)*100
    scorecard.loc['edge'] = df.result.mean()
    scorecard.loc['noise'] = df.y_pred.diff().abs().mean()

    # derived metrics
    scorecard.loc['y_true_chg'] = df.y_true.abs().mean()
    scorecard.loc['y_pred_chg'] = df.y_pred.abs().mean()
    scorecard.loc['prediction_calibration'] = scorecard.loc['y_pred_chg']/scorecard.loc['y_true_chg']
    scorecard.loc['capture_ratio'] = scorecard.loc['edge']/scorecard.loc['y_true_chg']*100

    # metrics for a subset of predictions
    scorecard.loc['edge_long'] = df[df.sign_pred == 1].result.mean()  - df.y_true.mean()
    scorecard.loc['edge_short'] = df[df.sign_pred == -1].result.mean()  - df.y_true.mean()

    scorecard.loc['edge_win'] = df[df.is_correct == 1].result.mean()  - df.y_true.mean()
    scorecard.loc['edge_lose'] = df[df.is_incorrect == 1].result.mean()  - df.y_true.mean()

    return scorecard    

y_pred_diff = pd.DataFrame(y_pred_scaled, columns=['y_pred']).diff()
y_pred_diff = y_pred_diff.set_index(pd.DatetimeIndex(y_test_dates_list)).dropna()
y_actual_diff = pd.DataFrame(y_actual_scaled, index=y_test_dates_list, columns=['y_true']).diff().dropna()
metric_df = make_metric_df(y_pred_diff, y_actual_diff)
calc_scorecard(metric_df)

In [34]:
def get_top_correlated_features(df, d):
    df_2 = df.fillna(0)
    corrs = []
    for col in df_2.columns[1:]:
        if df_2.dtypes[col] == 'float64' or df_2.dtypes[col] == 'int64':
            #calculate Pearson's correlation
            corr, _ = pearsonr(df_2[col], df_2['Price'])
            corrs.append(corr)
            print(f"{col} : {corr}")
    coeff_df = pd.DataFrame(corrs, columns = ['PC'], index=df_2.columns[1:])
    return coeff_df.abs().where(coeff_df['PC']>0.5).dropna().where(coeff_df['PC'] != 1).dropna().sort_values('PC', ascending=False).head(d).index

diffed = df
for col in diffed.columns:
    if diffed.dtypes[col] == 'float64' or diffed.dtypes[col] == 'int64':
        diffed[col] = diffed[col].diff().fillna(0)>0
#diffed.head()
#get_top_correlated_features(diffed, 10)    
diffed.corr()['Price'].sort_values().tail(15)

TweetCount            0.038973
toSum_84f3            0.039174
fromSum_6853          0.041674
from_tx_count_ab89    0.043112
toSum_e428            0.043733
to_tx_count_fdc0      0.044008
toSum_fdc0            0.044008
fromSum_fdc0          0.044508
from_tx_count_8258    0.045686
fromSum_ab89          0.060846
fromSum_92ff          0.061653
fromSum_92fe          0.072640
from_tx_count_6853    0.076699
Price                 1.000000
MarketCap             1.000000
Name: Price, dtype: float64

In [ ]:
def MDALoss(yTrue,yPred):
    # mean directional accuracy: 1/N Sum(I(sign(A_t - A_t-1) == sign(F_t - A_t-1))
    # A: actual, F: predicted, N: data points
    # where I is the indicator function
    # MDA = mean(max(A_t - A_t-1, 0))
    Z = K.equal(K.sign(yTrue[1:]-yTrue[:-1]), K.sign(yPred[1:]-yPred[:-1]))
    L = -K.mean(K.cast(Z, 'float32'))
    return L

In [ ]:
def recover_original_from_diff(diffed, first_element_original):
    z = first_element_original + diffed.cumsum().reset_index(drop=True).fillna(0)
    z = z.append(pd.Series(name='blah')).shift()
    z['val'].loc[0] = first_element_original
    return z.reset_index(drop=True)